# Experiments

The goal of this notebook is to implement different components of the agent to understand how they should be implemented.

In [ ]:
# Import libraries
import os
from enum import Enum
from pydantic import BaseModel, Field
from typing import Annotated, Dict, List, Optional, Union, Literal, Any

from IPython.display import Image, display
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langchain_core.prompts import PromptTemplate
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END, add_messages
from langchain_core.messages import SystemMessage, HumanMessage, AnyMessage

from src.config.settings import settings

In [ ]:
# Set langsmith project
os.environ["LANGSMITH_API_KEY"] = settings.langsmith_api_key.get_secret_value()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = settings.app_name

## Research Agent

In [ ]:
# State
class CampaignGoal(str, Enum):
    """Possible objectives for an advertising campaign"""
    AWARENESS = 'awareness'
    TRAFFIC = 'traffic'
    ENGAGEMENT = 'engagement'
    LEADS = 'leads'
    APP_INSTALLS = 'app installs'


class AdPlatform(str, Enum):
    """Specific ad placements across different social platforms"""
    INSTAGRAM_REELS = "instagram_reels"
    INSTAGRAM_STORY = "instagram_video_story"
    FACEBOOK_FEEDS = "facebook_feed"
    FACEBOOK_STORY = "facebook_story"
    YOUTUBE_SHORT = "youtube_short"
    TIKTOK_REELS = "tiktok_reels"


class AudienceGender(str, Enum):
    """
    Gender identity options for the target audience.
    """
    MALE = "male"
    FEMALE = "female"
    ALL = "male and female"


class AudienceLocation(str, Enum):
    """
    Location options for the target audience.
    """
    USA = "united states"
    UK = "united kingdom"
    CANADA = "canada"
    AUSTRALIA = "australia"


class AudienceIncomeRange(str, Enum):
    """
    Income range categories for audience segmentation.
    """
    LOW = "<$30k"
    LOWER_MIDDLE = "$30k–$60k"
    MIDDLE = "$60k–$100k"
    UPPER_MIDDLE = "$100k–$200k"
    HIGH = ">$200k"


class AudienceAgeRange(BaseModel):
    min_age: int = Field(
        ...,
        description="Minimum age for the target audience"
    )
    max_age: int = Field(
        ...,
        description="Maximum age for the target audience"
    )


class ProductPlatform(str, Enum):
    """Mobile platforms supported by the product."""
    ios = "ios"
    android = "android"
    web="web"


class Product(BaseModel):
    """Product information"""
    name: str = Field(
        ...,
        description="Name of the product"
    )
    description: str = Field(
        ...,
        description="A brief overview of the product"
    )
    features: Dict[str, str] = Field(
        ...,
        description="Dictionary mapping feature names to their description"
    )
    supported_platforms: List[ProductPlatform] = Field(
        ...,
        description="Supported platforms for the application"
    )

In [ ]:
# Audience researcher output schema
class AudienceProfile(BaseModel):
    core_values: List[str] = Field(
        ...,
        description="The audience’s fundamental beliefs and priorities (e.g., privacy, convenience, achievement)."
    )
    daily_routines: List[str] = Field(
        ...,
        description="Typical daily habits, recurring activities, or schedules common for this audience."
    )
    behavioral_patterns: List[str] = Field(
        ...,
        description="Frequent actions and tendencies in digital, social, or shopping behaviors."
    )
    pain_points: List[str] = Field(
        ...,
        description="Persistent frustrations, challenges, and unmet needs or desires for this group."
    )
    emotional_triggers: List[str] = Field(
        ...,
        description="Words, scenarios, or experiences that evoke strong emotions or reactions."
    )
    decision_process: List[str] = Field(
        ...,
        description="Typical steps, questions, and criteria used by the audience to make product or app choices."
    )
    summary: Optional[str] = Field(
        ...,
        description="Brief natural language summary highlighting the overall persona and mindset."
    )

In [ ]:
class GraphState(BaseModel):
    messages: Annotated[List[AnyMessage], add_messages] = Field(
        [],
        description="Messages for audience research agent"
    )
    campaign_goal: CampaignGoal = Field(
        ...,
        description="Goal of the marketing campaign"
    )
    ad_platform: AdPlatform = Field(
        ...,
        description="Platform for the advertisement"
    )
    audience_gender: AudienceGender = Field(
        ...,
        description="Gender of the target audience"
    )
    audience_location: AudienceLocation = Field(
        ...,
        description="Location of the target audience"
    )
    audience_income_range: AudienceIncomeRange = Field(
        ...,
        description="Income range for the target audience"
    )
    audience_age_range: AudienceAgeRange = Field(
        ...,
        description="Age range for the target audience"
    )
    audience_interests: Optional[List[str]] = Field(
        None,
        description="List of interests for the target audience"
    )
    product_details: Product = Field(
        ...,
        description="Details about the product"
    )
    audience_profile: Optional[AudienceProfile] = Field(
        None,
        description="Result of audience researcher"
    )


In [ ]:
audience_research_system_message = PromptTemplate(
    template="""You are an expert audience researcher, specialized in transforming basic demographics into rich psychological profiles for ad copywriting teams. Your job is to discover actionable insights about the audience’s core values, daily routines, behavioral patterns, pain points, emotional triggers, and decision-making processes.

You have access to a websearch tool. The effectiveness of your research depends on crafting intelligent, targeted search queries—this is critical for high-quality results.

You may use the search tool up to 5 times per task. Plan your queries carefully to maximize relevance and depth.

**Goal:**
Based on your findings, fill out every field of the AudienceProfile schema with clear, evidence-based information. Favor concise, actionable lists and summaries that copywriters can use directly.

**Instructions:**
- Start by reviewing the context provided by the user about the campaign and audience.
- For each insight required (values, routines, patterns, pain points, emotional triggers, decision steps), create focused search queries and process results.
- Synthesize your findings; do not copy/paste raw information from the web—always interpret and rephrase for clarity and usefulness.
- Limit websearch tool calls to 5; prioritize quality over quantity.

**Output Schema:**
return your answer strictly using the 'build_audience_profile' tool.
"""
)

audience_research_user_message = PromptTemplate(
    input_variables=["campaign_goal", "ad_platform", "audience_gender", "audience_location", "audience_income_range", "audience_age_range", "audience_interests", "product_details", "product_platform"],
    template="""Here is the context for your audience research task. Use these details to guide your information gathering and ensure your psychological profile is highly relevant.

- **Campaign Goal:** {campaign_goal}
- **Ad Platform:** {ad_platform}
- **Audience Gender:** {audience_gender}
- **Audience Location:** {audience_location}
- **Audience Income Range:** {audience_income_range}
- **Audience Age Range:** {audience_age_range}
- **Audience Interests:** {audience_interests}
- **Product Platform:** {product_platform}

**Product(Digital Service) Details:**
Name: {product_details.name}
Description: {product_details.description}
Features: {product_details.features}
Supported Platforms: {product_details.supported_platforms}
"""
)

In [ ]:
# Create LLM and bind tool
llm = ChatOpenAI(api_key=settings.open_ai_api_key.get_secret_value(), model="gpt-4o")

In [ ]:
search_tool = TavilySearch(tavily_api_key=settings.tavily_api_key.get_secret_value(), max_results=3)

In [ ]:
# Nodes
def audience_research_node(state: GraphState):
    messages = state.audience_research_messages
    campaign_goal = state.campaign_goal
    ad_platform = state.ad_platform
    audience_gender = state.audience_gender
    audience_location = state.audience_location
    audience_income_range = state.audience_income_range
    audience_age_range = state.audience_age_range
    audience_interests = state.audience_interests if state.audience_interests is not None else []
    product_details = state.product_details
    product_platform = product_details.supported_platforms

    # Bind tools to the LLM
    llm_with_tools = llm.bind_tools([search_tool])
    # Specify the structured output
    audience_researcher_llm = llm_with_tools.with_structured_output(AudienceProfile)

    # Create prompt
    system_message = SystemMessage(content=audience_research_system_message.format())
    user_message = HumanMessage(content=audience_research_user_message.format(campaign_goal=campaign_goal, ad_platform=ad_platform, audience_gender=audience_gender, audience_location=audience_location, audience_income_range=audience_income_range, audience_age_range=audience_age_range, audience_interests=audience_interests, product_details=product_details, product_platform=product_platform))

    # Create messages
    if len(messages) == 0:
        messages.append(system_message)
        messages.append(user_message)

    ai_message = audience_researcher_llm.invoke(messages)

    return {"audience_profile": getattr(ai_message, "content", ai_message), "audience_research_messages": messages}


In [ ]:
# Condition
def tools_condition(
    state: Union[list[AnyMessage], dict[str, Any], BaseModel],
    messages_key: str = "audience_research_messages",
) -> Literal["tools", "__end__"]:
    if isinstance(state, list):
        ai_message = state[-1]
    elif isinstance(state, dict) and (messages := state.get(messages_key, [])):
        ai_message = messages[-1]
    elif messages := getattr(state, messages_key, []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "__end__"


# Graph
audience_research_builder = StateGraph(GraphState)

audience_research_builder.add_node("audience_research", audience_research_node)
audience_research_builder.add_node("tools", ToolNode([search_tool]))

audience_research_builder.add_edge(START, "audience_research")
audience_research_builder.add_conditional_edges("audience_research", tools_condition)
audience_research_builder.add_edge("tools", "audience_research")

memory = InMemorySaver()
audience_research_agent = audience_research_builder.compile(checkpointer=memory)

display(Image(audience_research_agent.get_graph().draw_mermaid_png()))

In [ ]:
thread = {"configurable": {"thread_id": 1}}
initial_state = GraphState(
    campaign_goal=CampaignGoal.AWARENESS,
    ad_platform=AdPlatform.INSTAGRAM_REELS,
    audience_gender=AudienceGender.ALL,
    audience_location=AudienceLocation.USA,
    audience_income_range=AudienceIncomeRange.LOWER_MIDDLE,
    audience_age_range=AudienceAgeRange(min_age=25, max_age=35),
    product_details=Product(
        name="Delisio- Your Personal Chef",
        description="Delisio is a personal chef and nutrition assistant. When a user signs up, they enter their age, weight, height, diet (e.g., vegan, keto), nutritional goal (e.g., weight loss, muscle gain), allergies, and equipments. Delisio then provides personalized recipes based on their preferences and taste preferences and uses AI to personalize user food experience",
        features={
            "Photo to Recipe": "Photo to Recipe: User uploads a photo of any dish, specify equipments, define calorie preference, and Delisio generates a personalized recipe tailored to their preferences",
            "Surprise Me": "User selects meal type, cuisine, specify equipments, define calorie preference; Delisio generates a unique, personalized recipe (e.g., a vegetarian Chinese breakfast)",
            "Nutrition Scanner": "User scans their food; Delisio analyzes and reports nutrients, vitamins, and minerals.",
        },
        supported_platforms=[ProductPlatform.ios, ProductPlatform.android]
    ),
)

In [ ]:
for event in audience_research_agent.stream(initial_state, config=thread, stream_mode="updates"):
    print(event)

In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command

# --- FIRST GRAPH ---

def node_a(state):
    messages = state.get("messages", [])
    messages.append("Node A visited")
    return Command(goto="node_b", update={"messages": messages})

def node_b(state):
    messages = state.get("messages", [])
    messages.append("Node B visited")
    return Command(goto=END, update={"messages": messages})

graph1 = StateGraph(MessagesState)
graph1.add_node(node_a)
graph1.add_node(node_b)
graph1.add_edge(START, "node_a")
graph1.add_edge("node_a", "node_b")
graph1.add_edge("node_b", END)
graph1_compiled = graph1.compile()

# Run first graph stand-alone
result1 = graph1_compiled.invoke({"messages": []})
# Output: ['Node A visited', 'Node B visited']

# --- SECOND GRAPH ---

def node1(state):
    messages = state.get("messages", [])
    messages.append("Second graph Node1")
    # Invoke first graph INSIDE node1
    sub_result = graph1_compiled.invoke({"messages": messages})
    # Take the final messages from subgraph
    return Command(goto="node2", update={"messages": sub_result["messages"]})

def node2(state):
    messages = state.get("messages", [])
    messages.append("Second graph Node2")
    return Command(goto=END, update={"messages": messages})

graph2 = StateGraph(MessagesState)
graph2.add_node(node1)
graph2.add_node(node2)
graph2.add_edge(START, "node1")
graph2.add_edge("node1", "node2")
graph2.add_edge("node2", END)
graph2_compiled = graph2.compile()

# Run second graph!
result2 = graph2_compiled.invoke({"messages": []})
# Output: ['Node A visited', 'Node B visited', 'Second graph Node2']


In [ ]:
from IPython.display import Image, display

# Show the agent
display(Image(graph2_compiled.get_graph(xray=True).draw_mermaid_png()))
